# Building A Zero-Data Model Foundry

## Setting Up

In [1]:
import time
import json
import os
import random
import shutil
from dataclasses import dataclass
from pathlib import Path
from uuid import uuid4

import covalent as ct
import covalent_cloud as cc
import torch
from covalent_cloud.cloud_executor.models.gpu import GPU_TYPE
from datasets import Dataset, load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

## Authenticating with Covalent Cloud

In [2]:
CC_API_KEY = os.environ["CC_API_KEY"]  # set in `environment.yml` file
cc.save_api_key(CC_API_KEY)

## Create a [cloud volume](https://docs.covalent.xyz/docs/cloud/guides/cloud_storage) for persistent storage

In [3]:
volume = cc.volume("model-storage")  # store fine-tuned models and generated datasets

## Create [runtime environments](https://docs.covalent.xyz/docs/cloud/guides/cloud_custom_environments) for tasks and services

An environment for fine-tuning models:

In [4]:
FT_ENV = "model-fine-tuning"  # assign unique name for referring to this env

cc.create_env(
    name=FT_ENV,
    pip=[
        "accelerate==0.29.1",
        "bitsandbytes==0.43.0",
        "datasets==2.18.0",
        "pandas==2.2.1",
        "scipy==1.12.0",
        "sentencepiece==0.2.0",
        "torch==2.2.2",
        "transformers==4.39.3",
        "trl==0.8.1",
        "tqdm==4.66.2",
        "peft==0.10.0",
    ],
    wait=True,
)

Environment Already Exists.


Another environment for running the data generator LLM:

In [5]:
DATA_ENV = "data-generation"
cc.create_env(name=DATA_ENV, pip=["vllm"], wait=True)

Environment Already Exists.


---

# Service: Data Generator LLM

This service hosts a powerful LLM that generates synthetic data for fine tuning another model.

<div align="center">
<img src="./assets/data-generator_dark_mode.png" alt="Highlight data generator component" height=550px/>
</div>

## Backend for the Data Generator LLM

- H100 GPU

- 48 GB RAM

In [6]:
data_generator_ex = cc.CloudExecutor(
    env=DATA_ENV,
    num_cpus=6,
    num_gpus=1,
    gpu_type=GPU_TYPE.H100,
    memory="48GB",
    time_limit="12 hours",
)

In [7]:
PROMPT_TEMPLATE = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
    "You are a knowledgeable assistant who generates fine-tuning data for an LLM. "
    "Please generate {target_items_per_response} data items for the fine-tuning task specified by the user.\n"
    "IMPORTANT: Return a JSON array of new items in the format: \"{return_format}\""
    "<|eot_id|>"
    "<|start_header_id|>user<|end_header_id|>{user_prompt}<|eot_id|>"
    "<|start_header_id|>assistant<|end_header_id|>"
)


@cc.service(executor=data_generator_ex, name="LLM Data Generator", auth=False)
def llm_data_generator(model_name):

    """Uses a powerful LLM to generate synthetic training data."""

    from vllm import LLM

    return {
        "llm": LLM(model=model_name, trust_remote_code=True, enforce_eager=True),
    }

@llm_data_generator.endpoint("/generate-data")
def generate_data(
    llm,
    task,
    return_format="[item] ## [label]",
    num_generations=5,
    target_items_per_response=5,
):
    """Generate data for a specific task."""

    from vllm import SamplingParams

    # Format task into prompt.
    prompt_ = PROMPT_TEMPLATE.format(
        target_items_per_response=target_items_per_response,
        return_format=return_format,
        user_prompt=json.dumps({
            "task": task,
            "constraint": "Respond with ONLY the generated data as a valid JSON array!",
        }),
    )

    def _seeded_sampling_params():
        seed = random.randint(0, 1_000_000)
        return SamplingParams(temperature=0.9, top_p=0.8, max_tokens=2000, seed=seed)

    # Create a batch of prompts and sampling params
    prompts_batch = [prompt_] * num_generations
    params_batch = [_seeded_sampling_params() for _ in range(num_generations)]

    # Generate data
    outputs = llm.generate(prompts_batch, params_batch)

    # Extract and filter generated data
    data_items = []
    for output in outputs:
        generated_text = output.outputs[0].text
        try:
            data_items.extend(json.loads(generated_text))
        except Exception:
            continue

    return data_items

---

# Service: New Fine-tuned Models

<div align="center">
<img src="./assets/finetune-service_dark_mode.png" alt="Highlight fine-tuned model component" height=550px/>
</div>

### Backend for the Fine-tuned Model Service

Run on:
- L40 GPU
- 48 GB RAM

In [8]:
ft_service_ex = cc.CloudExecutor(
    env=FT_ENV,
    num_cpus=25,
    num_gpus=1,
    gpu_type=GPU_TYPE.L40,
    memory="48GB",
    time_limit="12 hours"
)

In [9]:
@cc.service(executor=ft_service_ex, volume=volume)
def finetuned_llm_service(ft_model_path):

    """Hosts a fine-tuned LLM for text generation."""

    ft_model_path_ = Path("/tmp") / Path(ft_model_path).name
    if ft_model_path_.exists():
        shutil.rmtree(ft_model_path_)
    shutil.copytree(ft_model_path, ft_model_path_)

    # Load and configure saved model
    model = AutoModelForCausalLM.from_pretrained(ft_model_path_, device_map="auto", do_sample=True)

    # Load and configure tokenizer
    tokenizer = AutoTokenizer.from_pretrained(ft_model_path_)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Combine model and tokenizer into a pipeline
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    return {"pipe": pipe, "model": model, "tokenizer": tokenizer}


@finetuned_llm_service.endpoint("/generate")
def generate_text(pipe, prompt, max_new_tokens=300):

    """Generate text from a prompt using the fine-tuned language model."""

    output = pipe(prompt, truncation=True, max_new_tokens=max_new_tokens, num_return_sequences=1)
    return output[0]["generated_text"]

---

# Service: Main Agent

<div align="center">
<img src="./assets/main-agent_dark_mode.png" alt="Highlight main agent component" height=550px/>
</div>

In [10]:
agent_ex = cc.CloudExecutor(env=FT_ENV, num_cpus=2, memory="12GB", time_limit="12 hours")

@cc.service(executor=agent_ex, name="Fine Tuning Launcher", auth=False, volume=volume)
def fine_tuning_launcher(lattice, data_generator_api):

    """Launches the fine-tuning workflow and deploys the fine-tuned model."""

    return {"finetune_lattice": lattice, "data_generator_api": data_generator_api}


@fine_tuning_launcher.endpoint("/submit", streaming=True)
def submit(
    finetune_lattice, data_generator_api,
    *,
    task,
    name,
    data_format="[item] ## [label]",
    num_generations=5,
    target_items_per_response=10,
    min_new_examples=2000,
    model_to_finetune="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
):
    """Receives a task description, generates fine-tuning data,
    and dispatches the fine-tuning + deployment workflow."""

    yield "Generating fine-tuning data "

    iteration = 1
    new_examples = []
    while len(new_examples) < min_new_examples:

        texts = data_generator_api.generate_data(
            task=task,
            return_format=data_format,
            num_generations=num_generations,
            target_items_per_response=target_items_per_response,
        )
        new_examples.extend(texts)

        yield "."
        iteration += 1

    yield f"\nGenerated {len(new_examples)} total examples\n"

    dataset_save_path = volume / f"data_{len(new_examples)}-{uuid4()}"
    yield f"Saving dataset at {dataset_save_path!s}\n"
    dataset = Dataset.from_dict({"text": new_examples})
    dataset_save_path.mkdir(parents=True, exist_ok=True)
    dataset.save_to_disk(dataset_save_path)

    cc.save_api_key(CC_API_KEY)

    yield "\nDispatching fine-tuning workflow\n"
    finetuned_llm_service.name = name
    dispatch_id = cc.dispatch(finetune_lattice, volume=volume)(
        model_to_finetune, str(dataset_save_path), finetuned_llm_service
    )
    yield f"Dispatch ID:\n{dispatch_id}\n"
    yield "Fine tuning new model "
    result = None
    while result is None:
        res = cc.get_result(dispatch_id)
        res.result.load()
        result = res.result.value
        time.sleep(10)
        yield "."

    yield f"\nNew Service ID: {result.function_id!s}\n"

---

# Workflow: Fine Tune & Deploy

This workflow runs model fine-tuning on a powerful GPU and deploys the model as a service.

<div align="center">
<img src="./assets/finetune-workflow_dark_mode.png" alt="Highlight fine-tune and deploy workflow" height=550px/>
</div>

## Training configuration params

This dataclass holds the myriad fine-tuning parameter defaults for the PEFT/LoRA approach.

In [11]:
@dataclass
class FineTuneArguments:
    # BitAndBytesConfig
    load_in_4bit: bool = True
    bnb_4bit_quant_type: str = "nf4"
    bnb_4bit_compute_dtype: str = "float16"
    bnb_4bit_use_double_quant: bool = False

    # TrainingArguments
    output_dir: str = "./outputs"
    learning_rate: float = 2e-3
    num_train_epochs: int = 5
    save_total_limit: int = 1
    save_strategy: str = "epoch"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 1
    optim: str = "paged_adamw_32bit"
    weight_decay: float = 0.001
    fp16: bool = False
    bf16: bool = False
    max_grad_norm: float = 0.3
    max_steps: int = -1
    warmup_ratio: float = 0.03
    group_by_length: bool = True
    lr_scheduler_type: str = "cosine"
    report_to: str = "none"

    # LoraConfig
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    r: int = 32
    bias: str = "none"
    task_type: str = "CAUSAL_LM"

    # SFTTrainer
    dataset_text_field: str = "text"
    max_seq_length: int = 1024
    packing: bool = True
    dataset_batch_size: int = 10

    @property
    def training_args(self):
        return TrainingArguments(
            output_dir=self.output_dir,
            num_train_epochs=self.num_train_epochs,
            per_device_train_batch_size=self.per_device_train_batch_size,
            gradient_accumulation_steps=self.gradient_accumulation_steps,
            optim=self.optim,
            save_strategy=self.save_strategy,
            save_total_limit=self.save_total_limit,
            learning_rate=self.learning_rate,
            weight_decay=self.weight_decay,
            fp16=self.fp16,
            bf16=self.bf16,
            max_grad_norm=self.max_grad_norm,
            max_steps=self.max_steps,
            warmup_ratio=self.warmup_ratio,
            group_by_length=self.group_by_length,
            lr_scheduler_type=self.lr_scheduler_type,
            report_to=self.report_to,
        )

    @property
    def lora_config(self):
        return LoraConfig(
            lora_alpha=self.lora_alpha,
            lora_dropout=self.lora_dropout,
            r=self.r,
            bias=self.bias,
            task_type=self.task_type,
        )

    @property
    def trainer_params(self):
        return {
            "dataset_text_field": self.dataset_text_field,
            "max_seq_length": self.max_seq_length,
            "packing": self.packing,
            "dataset_batch_size": self.dataset_batch_size,
        }

## Electrons (i.e. workflow tasks)

### Fine-tuning task

Run on:
- A100 GPU
- 32 GB RAM

Tasks exit and release resources after completion.

In [12]:
fine_tune_ex = cc.CloudExecutor(
    env=FT_ENV,
    num_cpus=6,
    num_gpus=1,
    gpu_type=GPU_TYPE.A100,
    memory="32GB",
    time_limit="6 hours"
)

In [13]:
@ct.electron(executor=fine_tune_ex)
def fine_tune_model_peft(model_path, dataset_path):

    """Run fine-tuning, save the model, and return the path to the saved model."""

    ft_args = FineTuneArguments()

    # Quantization configuration
    quant_config = BitsAndBytesConfig(
        load_in_4bit=ft_args.load_in_4bit,
        bnb_4bit_quant_type=ft_args.bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=getattr(torch, ft_args.bnb_4bit_compute_dtype),
        bnb_4bit_use_double_quant=ft_args.bnb_4bit_use_double_quant,
    )

    # Load and configure the downloaded model from pretrained
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=quant_config,
        device_map="auto",
        do_sample=True,
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Load and configure the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Load dataset
    dataset_path_ = Path("/tmp") / Path(dataset_path).name
    shutil.copytree(dataset_path, dataset_path_)
    dataset_path = dataset_path_
    dataset = load_from_disk(dataset_path, keep_in_memory=True)

    # Set up supervised fine-tuning trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=ft_args.lora_config,
        tokenizer=tokenizer,
        args=ft_args.training_args,
        **ft_args.trainer_params,
    )

    # Run training
    trainer.train()

    # Save trained model
    new_model_path = volume / (model_path.split("/")[-1] + f"_{uuid4()}")
    trainer.model.save_pretrained(new_model_path)
    trainer.tokenizer.save_pretrained(new_model_path)

    return new_model_path

## Workflow: Fine tuning (launched by Main Agent)

In [14]:
cpu_ex = cc.CloudExecutor(env=FT_ENV, num_cpus=2, memory="12GB")

@ct.lattice(executor=cpu_ex, workflow_executor=cpu_ex)
def finetune_workflow(model_id, data_path, llm_service):

    """Run fine tuning, then deploy the fine tuned model."""

    ft_model_path = fine_tune_model_peft(model_id, data_path)
    service_info = llm_service(ft_model_path)

    return service_info

---

# Workflow: Setting up The Zero-Data Model Foundry

In [15]:
@ct.lattice(executor=cpu_ex, workflow_executor=cpu_ex)
def ai_foundry_setup_workflow(ft_workflow, data_generator_model="unsloth/llama-3-8b-Instruct"):

    """Set up the data generator LLM and the launcher service."""

    data_generator_handle = llm_data_generator(data_generator_model)
    main_agent = fine_tuning_launcher(ft_workflow, data_generator_handle)
    return main_agent

In [16]:
dispatch_id = cc.dispatch(ai_foundry_setup_workflow, volume=volume)(ft_workflow=finetune_workflow)

print("Workflow Dispatch ID: ", dispatch_id)

res = cc.get_result(dispatch_id, wait=True)
res.result.load()
main_agent = res.result.value

print("Main Agent Service ID: ", main_agent.function_id)

Output()

Workflow Dispatch ID:  d9938b2b-246a-4a1e-b7ec-2fbd652f8ce3
Main Agent Service ID:  66fc01be50e1b9bb8cb4cb8b


---

# Invoking the Agent API

In [17]:
main_agent = cc.get_deployment(main_agent.function_id)
print(main_agent)

╭────────────────────────────── Deployment Information ───────────────────────────────╮
│  Name          Fine Tuning Launcher                                                 │
│  Description   Launches the fine-tuning workflow and deploys the fine-tuned model.  │
│  Function ID   66fc01be50e1b9bb8cb4cb8b                                             │
│  Address       https://fn-a.prod.covalent.xyz/66fc01be50e1b9bb8cb4cb8b              │
│  Status        ACTIVE                                                               │
│  Tags                                                                               │
│  Auth Enabled  No                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────╯
╭──────────────────────────────────────────────────────────────────────────╮
│                               POST /submit                               │
│  Streaming    Yes                                   

## Example 1: Spoiler Detector

In [18]:
task = "Fine-tuning an LLM to detect whether or not a movie review contains a spoiler."
for k in main_agent.submit(
    task=task,
    name="Fine-Tuned Spoiler Detector",
    min_new_examples=1000
):
    print(k.decode(), end="")

Generating fine-tuning data ........................
Generated 1000 total examples
Saving dataset at /volumes/model-storage/data_1000-efa18179-3651-411d-ba16-c39b374e4d6e

Dispatching fine-tuning workflow
Dispatch ID:
63364307-0602-43a0-9fc8-2ef78c6ba874
Fine tuning new model .......................................................................................................................
New Service ID: 66fc14ad50e1b9bb8cb4cb8e


In [19]:
# Load a client for the new spoiler agent
spoiler_agent = cc.get_deployment("66fc14ad50e1b9bb8cb4cb8e")

In [20]:
print(spoiler_agent)

╭──────────────────────── Deployment Information ─────────────────────────╮
│  Name          Fine-Tuned Spoiler Detector                              │
│  Description   Hosts a fine-tuned LLM for text generation.              │
│  Function ID   66fc14ad50e1b9bb8cb4cb8e                                 │
│  Address       https://fn-c.prod.covalent.xyz/66fc14ad50e1b9bb8cb4cb8e  │
│  Status        ACTIVE                                                   │
│  Tags                                                                   │
│  Auth Enabled  Yes                                                      │
╰─────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────────────────────────────────────────────────╮
│                                 POST /generate                                  │
│  Streaming    No                                                                │
│  Description  Generate text from a prompt using the fine-tuned

In [21]:
spoiler_agent.generate(
    prompt=(
        "This ripping action-adventure features stellar effects and a "
        "superb lead performance from Owen Teague as a timid simian "
        "who must rescue his clan from the clutches of a warlike tribe. ##"
    )
)
# Review of Kingdom of the Planet of the Apes (2024). Scored 90/100. [source: Metacritic]

'This ripping action-adventure features stellar effects and a superb lead performance from Owen Teague as a timid simian who must rescue his clan from the clutches of a warlike tribe. ## NOT A SPOILER'

In [22]:
spoiler_agent.generate(prompt="Add a lot of dull acting -- except Sir Ian McKellen and Andy Serkis -- and you have an uneven movie with yawns aplenty. ##").split("##")[-1].strip()
# Review of The Lord of the Rings: The Return of the King (2003). Scored 0/100. [source: Metacritic]

'No Spoiler'

In [23]:
spoiler_agent.generate(prompt="In the end, turns out Soylent green was people! Gross! ##").split("##")[-1].strip()

'spoiler'

In [24]:
spoiler_agent.generate(prompt="So the Planet of The Apes was Earth all along. That's a lame twist. ##").split("##")[-1].strip()

'spoiler'

## Example 2: Grammar Corrector

In [25]:
task = "Examples of bot responses that correct grammatical errors."
data_format = '"<|user|>{input_sentence}</s><|assistant|>{corrected_sentence}"'

for k in main_agent.submit(
    task=task,
    name="Fine-Tuned Grammar Corrector",
    data_format=data_format
):
    print(k.decode(), end="")

Generating fine-tuning data ........................................
Generated 2010 total examples
Saving dataset at /volumes/model-storage/data_2010-ae48f9c4-358c-4e56-9373-613fee904878

Dispatching fine-tuning workflow
Dispatch ID:
5af40b24-52fe-48c2-9e91-a6febed0e14e
Fine tuning new model ......................................................................................................................................
New Service ID: 66fc1d7750e1b9bb8cb4cb91


In [26]:
grammar_agent = cc.get_deployment("66fc1d7750e1b9bb8cb4cb91")

In [27]:
prompt = "<|user|>{}</s><|assistant|>"
def correct_grammar(sentence):
    prompt_ = prompt.format(sentence)
    response = grammar_agent.generate(prompt=prompt_).split("<|assistant|>")[-1].strip()
    print(response)

correct_grammar("I should of never got a pet.")  # should've
correct_grammar("Jerry and me argued about it.")  # Jerry and I
correct_grammar("He said, 'No cat bites it's own tail.'")  # its
correct_grammar("But if I had to choose, Id rather get a dog then a cat.")  # I'd, than
correct_grammar("All dogs bite they're own tails.")  # their

I should have never gotten a pet.
Jerry and I argued about it.
He said, 'No cat bites its own tail.'
If I had to choose, I would rather get a dog than a cat.
All dogs bite their own tails.


## Example 3: Emoji translation

In [28]:
task = "Fine-tuning an LLM to translate a sentence without any emojis into a string of only emojis with roughly the same meaning."
for k in main_agent.submit(
    task=task,
    name="Fine-Tuned Emoji Translator",
    data_format="[sentence] | [matching emoji string]"
):
    print(k.decode(), end="")

Generating fine-tuning data ....................................................
Generated 2040 total examples
Saving dataset at /volumes/model-storage/data_2040-d173489b-95e5-4752-99e6-42e21435ad07

Dispatching fine-tuning workflow
Dispatch ID:
44e98935-19bb-427e-b96c-019317469fc9
Fine tuning new model .......................................................................................................................
New Service ID: 66fc231550e1b9bb8cb4cb94


In [29]:
emoji_agent = cc.get_deployment("66fc231550e1b9bb8cb4cb94")

In [30]:
emoji_agent.generate(prompt="Swing dancing with my cat | ")

'Swing dancing with my cat | 🐱💃'

In [31]:
emoji_agent.generate(prompt="Saved up for a brand new car | ")

'Saved up for a brand new car | 🚗💸'

In [32]:
emoji_agent.generate(prompt="Paint me a scenic mountain, Mr. Ross | ")

'Paint me a scenic mountain, Mr. Ross | 🏔️🌲'

In [33]:
emoji_agent.generate(prompt="Let's grab some burgers on the way home | ")

"Let's grab some burgers on the way home | 🍔👌"

In [34]:
emoji_agent.generate(prompt="Starting my vacation tomorrow! | ")

'Starting my vacation tomorrow! | 🏖️🌴'